In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

df1 = pd.read_csv('tesla_train.csv', dtype={'end':str})
df2 = pd.read_csv('starbucks_train.csv', dtype={'end':str})
df3 = pd.read_csv('verizon_train.csv', dtype={'end':str})
df = pd.concat([df1, df2, df3])
print len(df1)
print len(df2)
print len(df3)
print len(df)

215
183
216
614


In [ ]:
df['y'] = df['y'] / 100
df['delta'] = df['delta'] / 100
df['delta_window'] = df['delta_window'] / 100
tmp = ['c_max_fa','c_max_re','c_max_fo','c_min_fa','c_min_re','c_min_fo','fa_avg','re_avg','fo_avg','c_big_fa','c_big_re','c_big_fo','c_small_fa','c_small_re','c_small_fo','c_max_fa_window','c_max_re_window','c_max_fo_window','c_min_fa_window','c_min_re_window','c_min_fo_window','fa_avg_window','re_avg_window','fo_avg_window','c_big_fa_window','c_big_re_window','c_big_fo_window','c_small_fa_window','c_small_re_window','c_small_fo_window']
for t in tmp:
    df[tmp] = np.log10(df[tmp] + 1)
df = shuffle(df).reset_index(drop=True)
df

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# X = df[['delta','delta_window']].values
# scaler = MinMaxScaler(feature_range=(0, 10))
# X = np.concatenate((X, scaler.fit_transform(df[['c_max_fa','c_max_re','c_max_fo','c_min_fa','c_min_re','c_min_fo','fa_avg','re_avg','fo_avg','c_big_fa','c_big_re','c_big_fo','c_small_fa','c_small_re','c_small_fo','c_max_fa_window','c_max_re_window','c_max_fo_window','c_min_fa_window','c_min_re_window','c_min_fo_window','fa_avg_window','re_avg_window','fo_avg_window','c_big_fa_window','c_big_re_window','c_big_fo_window','c_small_fa_window','c_small_re_window','c_small_fo_window']].values)), axis=1)
# scaler = MinMaxScaler(feature_range=(-10,10))
# X = np.concatenate((X, scaler.fit_transform(df[['c_max', 'c_w_max','c_min', 'c_w_min','c_avg', 'c_w_avg','c_med','c_big_c','c_small_c','fa_big_c','re_big_c','fo_big_c','c_max_window', 'c_w_max_window','c_min_window', 'c_w_min_window','c_avg_window', 'c_w_avg_window','c_med_window','c_big_c_window','c_small_c_window','fa_big_c_window','re_big_c_window','fo_big_c_window']].values)), axis=1)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df = df.drop(['end'], axis=1).values
# df = scaler.fit_transform(df)

# X = df[:, 1:]
# y = df[:, 0]

In [ ]:
df.keys()

In [ ]:
# X = df.drop(['end', 'y'], axis=1).values
X = df[['delta','delta_window']].values
# X = df[['delta','delta_window','c_max','c_min','c_avg','c_med','c_max_window','c_min_window','c_avg_window','c_med_window']].values
y = df['y'].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# pd.DataFrame(X)

In [ ]:
tmp = X.shape[0] / 6
X_test, y_test = X[-tmp:, :], y[-tmp:]
X, y = X[:-tmp, :], y[:-tmp]

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
plt.hist(y, bins='auto')
plt.show()

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression(fit_intercept=False)
k_fold = KFold(n_splits=5)
mses = []
accuracies = []
for k, (train, test) in enumerate(k_fold.split(X, y)):
    regr.fit((X[train]), y[train])
    coef = regr.coef_
    y_pred = regr.predict((X[test]))
    mse = mean_squared_error(y[test], y_pred)
    mses.append(mse)
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(y[test], y_pred)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    accuracies.append(accuracy)
    r2 = r2_score(y[test], y_pred)
    
print('\n5-CV MSE = {0:.4f} Accuracy = {1:.4f}\n'.format(np.mean(mses), np.mean(accuracies)))

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='linear', C=10.0)
k_fold = KFold(n_splits=5)
mses = []
accuracies = []
for k, (train, test) in enumerate(k_fold.split(X, y)):
    svr.fit((X[train]), y[train])
    y_pred = svr.predict((X[test]))
    mse = mean_squared_error(y[test], y_pred)
    mses.append(mse)
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(y[test], y_pred)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    accuracies.append(accuracy)
    r2 = r2_score(y[test], y_pred)
    
print('\n5-CV MSE = {0:.4f} Accuracy = {1:.4f}\n'.format(np.mean(mses), np.mean(accuracies)))

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(20,20),max_iter=500, learning_rate='adaptive', learning_rate_init=0.001, alpha=0.0001, solver='adam') # 20 20 0.3486
k_fold = KFold(n_splits=5)
mses = []
accuracies = []
for k, (train, test) in enumerate(k_fold.split(X, y)):
    mlp.fit(X[train], y[train])
    y_pred = mlp.predict(X[test])
    mse = mean_squared_error(y[test], y_pred)
    mses.append(mse)
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(y[test], y_pred)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    accuracies.append(accuracy)
    r2 = r2_score(y[test], y_pred)
    
print('\n5-CV MSE = {0:.4f} Accuracy = {1:.4f}\n'.format(np.mean(mses), np.mean(accuracies)))

In [ ]:
import xgboost as xgb

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(labels, preds)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    return 'my-error', 1 - accuracy

param = {}
param['objective'] = 'reg:linear'
param['verbosity'] = 0
param['eta'] = 0.01
param['max_depth'] = 2
param['min_child_weight'] = 4
param['subsample'] = 0.6
param['colsample_bytree'] = 0.6

k_fold = KFold(n_splits=5)
mses = []
accuracies = []
for k, (train, test) in enumerate(k_fold.split(X, y)):
    train_X = X[train]
    train_Y = y[train]
    test_X = X[test]
    test_Y = y[test]
    xg_train = xgb.DMatrix(train_X, label=train_Y)
    xg_test = xgb.DMatrix(test_X, label=test_Y)
    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 50
    bst = xgb.train(param, xg_train, num_round, watchlist, feval=evalerror)
    y_pred = bst.predict(xg_test)
    mse = mean_squared_error(y[test], y_pred)
    mses.append(mse)
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(y[test], y_pred)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    accuracies.append(accuracy)
    r2 = r2_score(y[test], y_pred)
    
print('\n5-CV MSE = {0:.4f} Accuracy = {1:.4f}\n'.format(np.mean(mses), np.mean(accuracies)))

In [ ]:
# XGB Grid Search

from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from xgboost import XGBRegressor
import pickle

def my_accuracy_func(y_true, y_pred):
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(y_true, y_pred)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    return accuracy

my_scorer = make_scorer(my_accuracy_func, greater_is_better=True)

params = {'learning_rate':[x * 0.01 for x in range(1,5)], 'min_child_weight':[4], 'gamma':[i/10 for i in range(5,40)],  'subsample':[0.6],
'colsample_bytree':[0.6], 'max_depth': [2,3,4]}

xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params, cv=5, scoring=my_scorer)
grid.fit(X, y)

In [ ]:
print grid.best_score_
print grid.best_params_
pd.DataFrame(grid.cv_results_)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    accuracy, count = 0, 0
    for i, (a, b) in enumerate(zip(labels, preds)):
        if a > 0.1 and b > 0.1: accuracy += 1
        if a < -0.1 and b < -0.1: accuracy += 1
        if b > 0.1 or b < -0.1: count += 1
    accuracy = accuracy * 1.0 / count
    return 'my-error', 1 - accuracy

param = {}
param['objective'] = 'reg:linear'
param['verbosity'] = 0
param['eta'] = 0.01
param['max_depth'] = 2
param['min_child_weight'] = 4
param['subsample'] = 0.6
param['colsample_bytree'] = 0.6

xg_train = xgb.DMatrix(X, label=y)
xg_test = xgb.DMatrix(X_test, label=y_test)
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 50
bst = xgb.train(param, xg_train, num_round, watchlist, feval=evalerror)
y_pred = bst.predict(xg_test)
mse = mean_squared_error(y_test, y_pred)
accuracy, count = 0, 0
for i, (a, b) in enumerate(zip(y_test, y_pred)):
    if a > 0.1 and b > 0.1: accuracy += 1
    if a < -0.1 and b < -0.1: accuracy += 1
    if b > 0.1 or b < -0.1: count += 1
accuracy = accuracy * 1.0 / count
    
print('\n5-CV MSE = {0:.4f} Accuracy = {1:.4f}\n'.format(mse, accuracy))